# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](https://github.com/GMDutta/lab-neural-networks/blob/master/your-code/tttboard.jpg?raw=1)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [3]:
# your code here

# Step 1: Data Engineering for Tic-Tac-Toe
# Step 1: Data Engineering for Tic-Tac-Toe using TensorFlow/Keras pipeline prep
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. Read CSV
df = pd.read_csv("tic-tac-toe.csv")

# 2. Inspect dataset
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nSample rows:")
print(df.head(10))
print("\nUnique counts per column:")
print(df.nunique())
print("\nClass distribution:")
print(df['class'].value_counts())

# 3. Convert categorical values to numeric
cell_map  = {'x': 1, 'o': -1, 'b': 0}
class_map = {'True': 1, 'False': 0, True: 1, False: 0}

cell_cols = df.columns[:-1]
target_col = df.columns[-1]

df_num = df.copy()
for c in cell_cols:
    df_num[c] = df_num[c].map(cell_map)
df_num[target_col] = df_num[target_col].map(class_map).astype(int)

# 4. Separate input and output
X = df_num.iloc[:, :-1].astype(float).values
y = df_num.iloc[:, -1].astype(int).values

# 5. Normalize input data
scaler = MinMaxScaler()        # maps -1→0, 0→0.5, 1→1
X_scaled = scaler.fit_transform(X)

# 6. Summary checks
print("\nAfter conversion and normalization:")
print("X_scaled shape:", X_scaled.shape)
print("y shape:", y.shape)
print("X_scaled min/max:", X_scaled.min(), X_scaled.max())
print("y unique values:", np.unique(y))

# Optional: save processed data for model training
np.save("X_scaled.npy", X_scaled)
np.save("y.npy", y)


Shape: (958, 10)
Columns: ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']

Sample rows:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
5  x  x  x  x  o  o  b  b  o   True
6  x  x  x  x  o  b  o  o  b   True
7  x  x  x  x  o  b  o  b  o   True
8  x  x  x  x  o  b  b  o  o   True
9  x  x  x  x  b  o  o  o  b   True

Unique counts per column:
TL       3
TM       3
TR       3
ML       3
MM       3
MR       3
BL       3
BM       3
BR       3
class    2
dtype: int64

Class distribution:
class
True     626
False    332
Name: count, dtype: int64

After conversion and normalization:
X_scaled shape: (958, 9)
y shape: (958,)
X_scaled min/max: 0.0 1.0
y unique values: [0 1]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [6]:
# your code here

# Tic-Tac-Toe — Step 1 + Step 2 (end-to-end)
# Trains a NN to predict if X wins.

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ---------- Data load + prep ----------
def load_or_prepare():
    # Prefer preprocessed arrays if present
    if os.path.exists("X_scaled.npy") and os.path.exists("y.npy"):
        X = np.load("X_scaled.npy")
        y = np.load("y.npy")
        return X, y

    # Else read raw CSV and preprocess
    csv_paths = [p for p in ["tic-tac-toe.csv", "./data/tic-tac-toe.csv"] if os.path.exists(p)]
    assert len(csv_paths) > 0, "Upload tic-tac-toe.csv to the working directory."
    df = pd.read_csv(csv_paths[0])

    # Map categorical → numeric
    cell_map  = {'x': 1, 'o': -1, 'b': 0}
    class_map = {'True': 1, 'False': 0, True: 1, False: 0}
    cell_cols = df.columns[:-1]
    target_col = df.columns[-1]

    df_num = df.copy()
    for c in cell_cols:
        df_num[c] = df_num[c].map(cell_map)
    df_num[target_col] = df_num[target_col].map(class_map).astype(int)

    X = df_num.iloc[:, :-1].astype(float).values
    y = df_num.iloc[:, -1].astype(int).values

    # Normalize inputs to [0,1]
    X = MinMaxScaler().fit_transform(X)

    # Cache
    np.save("X_scaled.npy", X)
    np.save("y.npy", y)
    return X, y

X, y = load_or_prepare()

# ---------- Train / test split ----------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---------- Model ----------
# 2-class softmax output; sparse labels (0/1) → sparse_categorical_crossentropy
model = keras.Sequential([
    layers.Input(shape=(9,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(2, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# ---------- Train ----------
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

# ---------- Evaluate ----------
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}  |  Test loss: {test_loss:.4f}")

model.save("tic-tac-toe.keras")
print("Saved model to tic-tac-toe.keras")

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4197 - loss: 0.7599 - val_accuracy: 0.6364 - val_loss: 0.6370
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6578 - loss: 0.6274 - val_accuracy: 0.6364 - val_loss: 0.6214
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6622 - loss: 0.6042 - val_accuracy: 0.6558 - val_loss: 0.5967
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6902 - loss: 0.5768 - val_accuracy: 0.7208 - val_loss: 0.5753
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7258 - loss: 0.5414 - val_accuracy: 0.7403 - val_loss: 0.5504
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7579 - loss: 0.5256 - val_accuracy: 0.7338 - val_loss: 0.5348
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7964 - loss: 0.4902 - val_accuracy: 0.7338 - val_loss: 0.5230
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7647 - loss: 0.4812 - val_accuracy: 0.7338 - val_loss

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [7]:
# your code here

# Step 3: Make Predictions with the saved model

import numpy as np
from tensorflow import keras

# Load test data (ensure these match your training preprocessing)
X = np.load("X_scaled.npy")
y = np.load("y.npy")

# Split same way as training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Load the saved model
model = keras.models.load_model("tic-tac-toe.keras")

# Select a few random rows
idx = np.random.choice(len(X_test), 5, replace=False)
X_sample = X_test[idx]
y_true = y_test[idx]

# Predict
pred_probs = model.predict(X_sample)
pred_classes = np.argmax(pred_probs, axis=1)

# Display results
for i in range(len(idx)):
    print(f"Row {i+1}: True={y_true[i]}, Pred={pred_classes[i]}, Prob={pred_probs[i]}")






1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Row 1: True=1, Pred=1, Prob=[0.06238251 0.93761754]
Row 2: True=0, Pred=0, Prob=[0.94828403 0.05171596]
Row 3: True=1, Pred=1, Prob=[0.1423307 0.8576693]
Row 4: True=0, Pred=0, Prob=[0.7959058  0.20409416]
Row 5: True=1, Pred=1, Prob=[4.5960967e-04 9.9954045e-01]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [8]:
# your code here

# Step 4: Improve the Tic-Tac-Toe model

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load preprocessed data
X = np.load("X_scaled.npy")
y = np.load("y.npy")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Custom optimizer with modified learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Deeper model (increased complexity)
model = keras.Sequential([
    layers.Input(shape=(9,)),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(2, activation="softmax")
])

# Compile
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train longer (e.g. 100 epochs)
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=1
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Improved Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

# Save improved model
model.save("tic-tac-toe-improved.keras")
print("Saved improved model to tic-tac-toe-improved.keras")


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6185 - loss: 0.6579 - val_accuracy: 0.6364 - val_loss: 0.6368
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6537 - loss: 0.6084 - val_accuracy: 0.6364 - val_loss: 0.6037
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6588 - loss: 0.5759 - val_accuracy: 0.6688 - val_loss: 0.5786
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7156 - loss: 0.5267 - val_accuracy: 0.7662 - val_loss: 0.5513
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7683 - loss: 0.5140 - val_accuracy: 0.7987 - val_loss: 0.5194
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7954 - loss: 0.4783 - val_accuracy: 0.7922 - val_loss: 0.4935
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7868 - loss: 0.4491 - val_accuracy: 0.7987 - val_loss: 0.4579
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8309 - loss: 0.4161 - val_accuracy: 0.8247 - 

**Which approach(es) did you find helpful to improve your model performance?**

In [9]:
# your answer here

# Tic-Tac-Toe: compact improvement script — depth, learning rate, epochs
# Trains several configs, evaluates, saves best model.

import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ---------------- Data ----------------
def load_data():
    if os.path.exists("X_scaled.npy") and os.path.exists("y.npy"):
        X = np.load("X_scaled.npy"); y = np.load("y.npy")
        return X, y
    # fallback: preprocess from CSV if present
    assert os.path.exists("tic-tac-toe.csv"), "Provide X_scaled.npy/y.npy or tic-tac-toe.csv"
    df = pd.read_csv("tic-tac-toe.csv")
    cell_map  = {'x': 1, 'o': -1, 'b': 0}
    class_map = {'True': 1, 'False': 0, True: 1, False: 0}
    cells, target = df.columns[:-1], df.columns[-1]
    for c in cells: df[c] = df[c].map(cell_map)
    df[target] = df[target].map(class_map).astype(int)
    X = MinMaxScaler().fit_transform(df.iloc[:, :-1].astype(float).values)
    y = df.iloc[:, -1].astype(int).values
    np.save("X_scaled.npy", X); np.save("y.npy", y)
    return X, y

X, y = load_data()
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ---------------- Model factory ----------------
def build_model(widths):
    m = keras.Sequential([layers.Input(shape=(9,))])
    for w in widths: m.add(layers.Dense(w, activation="relu"))
    m.add(layers.Dense(2, activation="softmax"))
    return m

# ---------------- Training utility ----------------
def train_eval(widths, lr, epochs):
    model = build_model(widths)
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    callbacks = [
        keras.callbacks.EarlyStopping(patience=12, restore_best_weights=True, monitor="val_accuracy"),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=6, verbose=0, monitor="val_loss")
    ]
    model.fit(
        Xtr, ytr, validation_split=0.2, epochs=epochs, batch_size=32, verbose=0, callbacks=callbacks
    )
    loss, acc = model.evaluate(Xte, yte, verbose=0)
    return model, acc, loss

# ---------------- Search a few good configs ----------------
configs = [
    {"widths":[64,64,32,16], "lr":0.001,  "epochs":120},
    {"widths":[128,64,32],   "lr":0.0005, "epochs":160},
    {"widths":[64,64,64,32], "lr":0.002,  "epochs":100},
    {"widths":[128,128,64],  "lr":0.001,  "epochs":150},
]

best = {"acc":-1, "loss":9, "model":None, "cfg":None}
for cfg in configs:
    model, acc, loss = train_eval(**cfg)
    print(f"cfg={cfg}  ->  acc={acc:.4f}  loss={loss:.4f}")
    if acc > best["acc"] or (acc == best["acc"] and loss < best["loss"]):
        best.update({"acc":acc, "loss":loss, "model":model, "cfg":cfg})

# ---------------- Save best ----------------
best["model"].save("tic-tac-toe-improved.keras")
print(f"BEST cfg={best['cfg']}  acc={best['acc']:.4f}  loss={best['loss']:.4f}")
print("Saved to tic-tac-toe-improved.keras")

# ---------------- Quick sanity predictions ----------------
idx = np.random.choice(len(Xte), 5, replace=False)
pred = np.argmax(best["model"].predict(Xte[idx], verbose=0), axis=1)
print("True:", yte[idx].tolist())
print("Pred:", pred.tolist())


cfg={'widths': [64, 64, 32, 16], 'lr': 0.001, 'epochs': 120}  ->  acc=0.9688  loss=0.1316
cfg={'widths': [128, 64, 32], 'lr': 0.0005, 'epochs': 160}  ->  acc=0.9844  loss=0.1174
cfg={'widths': [64, 64, 64, 32], 'lr': 0.002, 'epochs': 100}  ->  acc=0.9740  loss=0.0957
cfg={'widths': [128, 128, 64], 'lr': 0.001, 'epochs': 150}  ->  acc=0.9740  loss=0.1172
BEST cfg={'widths': [128, 64, 32], 'lr': 0.0005, 'epochs': 160}  acc=0.9844  loss=0.1174
Saved to tic-tac-toe-improved.keras
True: [1, 1, 1, 0, 1]
Pred: [1, 1, 1, 0, 1]
